# **Ciclo de Vida de Creación y Entrenamiento de Modelos en Pipelines**


### **1. Ingesta de Datos (1 hora)**

#### **1.1. Enfoque de Azure ML para la Ingesta de Datos**
Azure Machine Learning ofrece múltiples opciones para la ingesta de datos, permitiendo integrar datos desde diversas fuentes externas. Estas opciones están diseñadas para ser escalables, seguras y fáciles de usar.

- **Azure Blob Storage:**  
  - Ideal para almacenar grandes volúmenes de datos no estructurados.
  - Proporciona conectores nativos para acceder a archivos como CSV, JSON, imágenes, etc.
  - Soporta autenticación mediante claves de acceso o tokens SAS (Shared Access Signature).

- **Bases de Datos SQL:**  
  - Permite conectar bases de datos SQL Server, PostgreSQL o MySQL utilizando drivers como `pyodbc` o `SQLAlchemy`.

- **APIs RESTful:**  
  - Integración con APIs externas para obtener datos en tiempo real.

- **Azure Data Lake Storage (ADLS):**  
  - Una alternativa más avanzada que Blob Storage, optimizada para análisis de big data.

- **Conjuntos de Datos Registrados en Azure ML:**  
  - Azure ML permite registrar conjuntos de datos (`Dataset`) para facilitar su reutilización en pipelines.

#### **Referencias:**
- [Azure Blob Storage Documentation](https://learn.microsoft.com/en-us/azure/storage/blobs/)
- [Azure Data Lake Storage Documentation](https://learn.microsoft.com/en-us/azure/storage/blobs/data-lake-storage-introduction)
- [Azure ML Datasets Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets)

#### **1.2. Opciones y Posibilidades**
- **Autenticación Segura:**  
  - Uso de Managed Identities o Azure Key Vault para manejar credenciales de forma segura.
- **Automatización:**  
  - Azure Data Factory puede usarse para automatizar la ingesta de datos desde múltiples fuentes.
- **Escalabilidad:**  
  - Azure Blob Storage y ADLS están diseñados para manejar grandes volúmenes de datos sin comprometer el rendimiento.

#### **Referencias:**
- [Azure Data Factory Documentation](https://learn.microsoft.com/en-us/azure/data-factory/)
- [Azure Key Vault Documentation](https://learn.microsoft.com/en-us/azure/key-vault/)

#### **1.3. Ejemplo Práctico: Ingesta de Datos desde Azure Blob Storage**
```python
from azure.storage.blob import BlobServiceClient
import pandas as pd

# Conectar al Blob Storage
blob_service_client = BlobServiceClient.from_connection_string("TU_CONNECTION_STRING")
blob_client = blob_service_client.get_blob_client(container="mi-contenedor", blob="mi-archivo.csv")

# Descargar archivo
with open("./datos/mi-archivo.csv", "wb") as my_blob:
    download_stream = blob_client.download_blob()
    my_blob.write(download_stream.readall())

# Cargar datos en un DataFrame
datos = pd.read_csv("./datos/mi-archivo.csv")
print(datos.head())
```

---

### **2. Preprocesamiento y Transformación (1.5 horas)**

#### **2.1. Conceptos Fundamentales**
El preprocesamiento es crucial para preparar los datos antes del entrenamiento del modelo. Azure ML proporciona herramientas para realizar estas tareas de manera eficiente.

- **Limpieza de Datos:**  
  - Manejo de valores faltantes (`fillna`, `dropna`).
  - Eliminación de duplicados y corrección de errores.

- **Normalización y Escalado:**  
  - Técnicas como `StandardScaler` (normalización Z-score) o `MinMaxScaler` (escalado entre 0 y 1).

- **Ingeniería de Características:**  
  - Creación de nuevas características mediante técnicas como PCA, one-hot encoding, o embeddings.

#### **Referencias:**
- [Scikit-learn Preprocessing Documentation](https://scikit-learn.org/stable/modules/preprocessing.html)
- [Azure ML Preprocessing Modules](https://learn.microsoft.com/en-us/azure/machine-learning/concept-preprocessing)

#### **2.2. Opciones y Posibilidades**
- **Módulos Predefinidos en Azure ML Designer:**  
  - Herramientas visuales para limpieza y transformación de datos sin necesidad de escribir código.
- **Scripts Personalizados:**  
  - Uso de Python para implementar transformaciones personalizadas.
- **Paralelización:**  
  - Procesamiento distribuido en clústeres para grandes volúmenes de datos.

#### **Referencias:**
- [Azure ML Designer Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/concept-designer)
- [Distributed Computing in Azure ML](https://learn.microsoft.com/en-us/azure/machine-learning/concept-compute-target)

#### **2.3. Ejemplo Práctico: Limpieza y Normalización de Datos**
```python
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Datos de ejemplo
datos = pd.DataFrame({
    'edad': [25, 30, None, 35],
    'ingreso': [50000, 60000, 70000, 80000]
})

# Limpieza: Rellenar valores faltantes
datos['edad'].fillna(datos['edad'].mean(), inplace=True)

# Normalización
scaler = StandardScaler()
datos_normalizados = scaler.fit_transform(datos)
print(datos_normalizados)
```

---

### **3. Entrenamiento del Modelo (2 horas)**

#### **3.1. Selección de Algoritmos**
Azure ML soporta una amplia gama de algoritmos de machine learning, tanto tradicionales como avanzados.

- **Regresión:**  
  - Regresión lineal, regresión logística, árboles de decisión.
- **Clasificación:**  
  - Random Forest, SVM, Gradient Boosting.
- **Clustering:**  
  - K-Means, DBSCAN, clustering jerárquico.

#### **Referencias:**
- [Scikit-learn Algorithms Documentation](https://scikit-learn.org/stable/supervised_learning.html)
- [Azure ML Algorithm Cheat Sheet](https://learn.microsoft.com/en-us/azure/machine-learning/algorithm-cheat-sheet)

#### **3.2. AutoML**
- Automatiza la selección de algoritmos, hiperparámetros y validación cruzada.
- Genera informes detallados sobre el rendimiento de cada modelo.

#### **Referencias:**
- [Azure AutoML Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/concept-automated-ml)

#### **3.3. Compute Instances**
- **Compute Targets:**  
  - Máquinas virtuales dedicadas, clústeres de GPU, Kubernetes.
- **Distribución:**  
  - Entrenamiento distribuido para acelerar procesos intensivos.

#### **Referencias:**
- [Azure Compute Targets Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/concept-compute-target)

#### **3.4. Ejemplo Práctico: Entrenamiento con AutoML**
```python
from azureml.train.automl import AutoMLConfig
from azureml.core import Experiment

# Configuración de AutoML
automl_config = AutoMLConfig(
    task="classification",
    primary_metric="accuracy",
    compute_target="mi-cluster",
    training_data=dataset,
    label_column_name="etiqueta",
    iterations=10
)

# Ejecutar experimento
experimento = Experiment(workspace=ws, name="mi-experimento-automl")
corrida = experimento.submit(automl_config)
corrida.wait_for_completion(show_output=True)
```

---

### **4. Evaluación del Modelo (1 hora)**

#### **4.1. Métricas Clave**
- **Clasificación:**  
  - Precisión, recall, F1-score, matriz de confusión.
- **Regresión:**  
  - RMSE, MAE, R².

#### **Referencias:**
- [Model Evaluation Metrics in Scikit-learn](https://scikit-learn.org/stable/modules/model_evaluation.html)
- [Azure ML Model Evaluation Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-track-experiments)

#### **4.2. Validación Cruzada**
- Divide los datos en k subconjuntos para evaluar la robustez del modelo.

#### **Referencias:**
- [Cross-Validation in Scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html)

#### **4.3. Ejemplo Práctico: Evaluación de un Modelo**
```python
from sklearn.metrics import classification_report

# Predicciones
y_pred = modelo.predict(X_test)

# Reporte de métricas
print(classification_report(y_test, y_pred))
```

---

### **5. Implementación en Pipelines (0.5 horas)**

#### **5.1. Diseño de Pipelines Modulares**
Azure ML Pipelines permite automatizar cada fase del ciclo de vida.

- **Pasos Comunes:**  
  - Ingesta, preprocesamiento, entrenamiento, evaluación.
- **Reutilización:**  
  - Pasos modulares que pueden reutilizarse en diferentes pipelines.

#### **Referencias:**
- [Azure ML Pipelines Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/concept-ml-pipelines)

#### **5.2. Ejemplo Práctico: Pipeline Completo**
```python
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

# Definir pasos
paso_ingesta = PythonScriptStep(script_name="ingesta.py", ...)
paso_preprocesamiento = PythonScriptStep(script_name="preprocesamiento.py", ...)
paso_entrenamiento = PythonScriptStep(script_name="entrenamiento.py", ...)
paso_evaluacion = PythonScriptStep(script_name="evaluacion.py", ...)

# Crear pipeline
pipeline = Pipeline(workspace=ws, steps=[paso_ingesta, paso_preprocesamiento, paso_entrenamiento, paso_evaluacion])
pipeline.validate()

# Ejecutar pipeline
experimento = Experiment(workspace=ws, name="mi-pipeline")
corrida = experimento.submit(pipeline)
```

---

### **Actividad Práctica (1 hora)**

**Título:** Crear un pipeline que ingrese datos, realice preprocesamiento, entrene un modelo de clasificación y evalúe su rendimiento.

#### **Pasos:**
1. **Ingesta de Datos:**  
   - Descargar un conjunto de datos desde Azure Blob Storage o una API RESTful.

2. **Preprocesamiento:**  
   - Limpiar los datos y realizar ingeniería de características.

3. **Entrenamiento:**  
   - Entrenar un modelo de clasificación utilizando Scikit-learn o AutoML.

4. **Evaluación:**  
   - Evaluar el modelo utilizando métricas clave como precisión y F1-score.

5. **Pipeline:**  
   - Diseñar un pipeline modular que automatice todas las etapas anteriores.

---

### **Conclusión**
Este módulo proporciona una visión completa del ciclo de vida de creación y entrenamiento de modelos dentro de pipelines. Al combinar teoría y práctica, los participantes estarán preparados para implementar soluciones de machine learning escalables y automatizadas en entornos reales.

Si tienes más preguntas o necesitas profundizar en algún tema, no dudes en preguntar. ¡Espero que este curso sea útil!

#### **Referencias Generales:**
- [Azure Machine Learning Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/)
- [Scikit-learn Documentation](https://scikit-learn.org/stable/documentation.html)